In [1]:
import sys; sys.path.insert(0, '../'); from lib.__init__ import *

*** THIS NOTEBOOK CAN BE USED TO ANALYSE RAW AND DEC OP.WVFs FROM LARSOFT ***

In [7]:
def convert_sampling_rate(wvf, sampling_rate, new_sampling_rate):
    time = np.arange(0, len(wvf), 1) / sampling_rate
    new_time = np.arange(0, len(wvf), 1) / new_sampling_rate
    new_wvf = np.interp(new_time, time, wvf)
    return new_wvf

def apply_smoothing(wvf, smoothing):
    return savgol_filter(wvf, smoothing, 3)

ciemat_template_dict_list = []
this_wvf = []
# ciemat_template_dict = {}
ciemat_templates_model = os.listdir("../template/CIEMAT")
# print(ciemat_templates_model)
for model in ciemat_templates_model:
    # print("Loading model: {}".format(model))
    ciemat_templates = os.listdir("../template/CIEMAT/"+model+"/")
    for ciemat_template in ciemat_templates:
        if "Laser" in ciemat_template:
            wvf_type = "LASER"
        if "SPE" in ciemat_template:
            wvf_type = "SPE"
        if "OV1" in ciemat_template:
            ov = 1
        if "OV2" in ciemat_template:
            ov = 2
        if "OV3" in ciemat_template:
            ov = 3
        
        # print("Loading template: {}".format(ciemat_template))
        convert = True
        if ".txt" in ciemat_template:
            short_wvf = np.loadtxt("../template/CIEMAT/"+model+"/"+ciemat_template)
            convert = False
        
        elif ".npz" in ciemat_template:
            short_wvf = np.load("../template/CIEMAT/"+model+"/"+ciemat_template, allow_pickle=True)["arr_0"][0]
            # print(this_wvf)

        elif ".root" in ciemat_template:
            root_file = uproot.open("../template/CIEMAT/"+model+"/"+ciemat_template)
            short_wvf = root_file[root_file.keys()[0]].to_numpy()[0]
            # print(this_wvf)
        
        else: continue

        if len(short_wvf) < len(this_wvf):
            this_wvf = np.concatenate((short_wvf,np.zeros(len(this_wvf)-len(short_wvf))))
            max_length = len(this_wvf)
        else:
            this_wvf = short_wvf
            max_length = len(this_wvf)
            # print(len(this_wvf))
            
        # this_wvf = apply_smoothing(this_wvf, 100)
        if convert:
            this_wvf = convert_sampling_rate(this_wvf, 1/4e9, 1/16e9)
        if np.argmax(this_wvf) != 100:
            this_wvf = np.roll(this_wvf,100-np.argmax(this_wvf))
        
        ciemat_template_dict = {"INST": "CIEMAT",
                                "NAME": ciemat_template,
                                "MODEL": model,
                                "OV": ov,
                                "AMP": np.max(this_wvf),
                                "TYPE": wvf_type,
                                "ADC":  this_wvf.tolist(),
                                "TIME": 16e-9*np.arange(len(this_wvf))}
        ciemat_template_dict_list.append(ciemat_template_dict)

# print(ciemat_template_dict)

In [8]:
ciemat_df = pd.DataFrame(ciemat_template_dict_list).explode(["ADC","TIME"])
fig = px.line(ciemat_df[ciemat_df["TYPE"]=="SPE"],x="TIME",y="ADC",color="NAME",facet_col="MODEL",facet_row="TYPE",title="CIEMAT Templates")
fig.update_layout(template="presentation")
fig.show()


In [9]:
milano_template_dict_list = []
milano_templates_model = os.listdir("../template/MILANO")
for model in milano_templates_model:
    milano_templates = os.listdir("../template/MILANO/"+model)
    for milano_template in milano_templates:
        print("Loading template: {}".format(milano_template))
        this_wvf = np.loadtxt("../template/MILANO/"+model+"/"+milano_template)
        this_wvf = np.concatenate((np.zeros(100),this_wvf))
        this_wvf = np.roll(this_wvf,100-np.argmax(this_wvf))
        # this_wvf = apply_smoothing(this_wvf, 25)

        milano_template_dict = {"INST": "MILANO",
                                "NAME": milano_template,
                                "MODEL": model,
                                "AMP": np.max(this_wvf),
                                "OV": 2,
                                "TYPE": "SPE",
                                "ADC":  this_wvf.tolist(),
                                "TIME": 16e-9*np.arange(len(this_wvf))}
        milano_template_dict_list.append(milano_template_dict)
milano_df = pd.DataFrame(milano_template_dict_list).explode(["ADC","TIME"])
fig = px.line(milano_df[milano_df["TYPE"]=="SPE"],x="TIME",y="ADC",color="NAME",facet_col="MODEL",facet_row="TYPE",title="Larsoft Templates")
fig.update_layout(template="presentation")
fig.update_xaxes(showline=True)
fig.show()

Loading template: DAPHNE2_FBK_2022.txt
Loading template: DAPHNE2_HPK_2022.txt


In [15]:
df = pd.concat([ciemat_df,milano_df])
fig = px.line(df[(df["TYPE"]=="SPE")],
                x="TIME",
                y="ADC",
                color="NAME",
                # color_discrete_sequence=["blue","red","green","orange","blue","brown","green","gray","orange"],
                # facet_row="INST",
                facet_col="MODEL",
                title="Templates Comparison"
                )
fig = format_coustom_plotly(fig,tickformat=(",.0s",",.0s"),tickmode=("auto","auto"))
fig.show()
# plotly.offline.plot(fig, filename='file.html')